In [16]:
%gui qt5
from starfish import Experiment, FieldOfView, display
from starfish.image import ApplyTransform, Filter, LearnTransform
from starfish.spots import FindSpots, DecodeSpots, AssignTargets
from starfish.types import Axes, FunctionSource

In [2]:
# import imaging_db.database.db_operations as db_ops
# import imaging_db.utils.db_utils as db_utils

# def get_n_z(dataset_id, credentials_str):

#     with db_ops.session_scope(credentials_str) as session:
#         frames = session.query(db_ops.Frames) \
#             .join(db_ops.FramesGlobal) \
#             .join(db_ops.DataSet) \
#             .filter(db_ops.DataSet.dataset_serial == dataset_id)
        
#         z_list = [f.slice_idx for f in frames]
        
#     return np.asarray(z_list)


In [3]:
# credentials_filename = '/Users/kevin.yamauchi/Documents/db_credentials_new.json'
# credentials_str = db_utils.get_connection_str(credentials_filename)

# image_ids = [
#     'ISP-2019-05-28-14-30-00-0001',
#     'ISP-2019-05-28-14-30-00-0002',
#     'ISP-2019-05-28-14-30-00-0003',
#     'ISP-2019-05-28-14-30-00-0004'
# ]

# for i in image_ids:
#     print(get_n_z(i, credentials_str))
    

In [4]:
# z = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
# dataset_id = 'ISP-2019-05-28-14-30-00-0004'

# with db_ops.session_scope(credentials_str) as session:
#     frames = session.query(db_ops.Frames) \
#         .join(db_ops.FramesGlobal) \
#         .join(db_ops.DataSet) \
#         .filter(db_ops.DataSet.dataset_serial == dataset_id)
#     frames2 = frames.filter(db_ops.Frames.slice_idx.in_(z))
#     z_list = [f.slice_idx for f in frames]
#     z_list2 = [f.slice_idx for f in frames2]
    
#     print(set(z_list))
#     print(set(z_list2))
    

In [1]:
from InSituToolkit.imaging_database import write_experiment

image_ids = [
    'ISP-2019-05-28-14-30-00-0001',
    'ISP-2019-05-28-14-30-00-0002',
    'ISP-2019-05-28-14-30-00-0003',
    'ISP-2019-05-28-14-30-00-0004',
]
z_slices = [
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
        [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
        [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
]
spot_channels = ['G', 'T', 'A', 'C']
stain_channel = ['FITC']
db_credentials = '/Users/kevin.yamauchi/Documents/db_credentials_new.json'
output_dir = "./data/iss"

write_experiment(db_credentials, output_dir, image_ids,
                 spot_channels = spot_channels, stain_channels=stain_channel,
                 positions = [3], z_slices = z_slices
)


In [9]:
# Load the data
exp = Experiment.from_json("./data/iss/experiment.json")

imgs = exp['fov_000'].get_images(FieldOfView.PRIMARY_IMAGES)
im = next(imgs)

In [10]:
exp

<starfish.Experiment (FOVs=1)>
{
fov_000: <starfish.FieldOfView>
  Primary Image: <slicedimage.TileSet (c: 4, r: 2, z: 13, x: 2048, y: 2048)>
  Auxiliary Images:
    stain: <slicedimage.TileSet (c: 1, r: 2, z: 13, x: 2048, y: 2048)>
}

In [17]:
display(im)

In [14]:
# Filter the image
ghp = Filter.GaussianHighPass(sigma=3)
high_passed = ghp.run(im, verbose=True, in_place=False)


glp = Filter.GaussianLowPass(sigma=1)
low_passed = glp.run(high_passed, in_place=False, verbose=True)

# Max intensity project the spots spots image
mproj = Filter.Reduce((Axes.ZPLANE,), func='max', module=FunctionSource.np)
mip = mproj.run(low_passed)

100%|██████████| 104/104 [00:00<00:00, 267.62it/s]


In [15]:
display(mip)

100%|██████████| 8/8 [00:00<00:00, 39.98it/s]


In [ ]:
# Display a single plane of data
single_plane = imgs.sel({Axes.ROUND: 0, Axes.CH: 0, Axes.ZPLANE: 0})


learn_translation = LearnTransform.Translation(reference_stack=dots, axes=Axes.ROUND, upsampling=1000)
transforms_list = learn_translation.run(imgs.reduce({Axes.CH, Axes.ZPLANE}, func="max"))
warp = ApplyTransform.Warp()
registered_imgs = warp.run(filtered_imgs, transforms_list=transforms_list, in_place=False, verbose=True)

In [ ]:
psd = DetectPixels.PixelSpotDecoder(
    codebook=experiment.codebook,
    metric='euclidean', # distance metric to use for computing distance between a pixel vector and a codeword
    norm_order=2, # the L_n norm is taken of each pixel vector and codeword before computing the distance. this is n
    distance_threshold=0.5176, # minimum distance between a pixel vector and a codeword for it to be called as a gene
    magnitude_threshold=1.77e-5, # discard any pixel vectors below this magnitude
    min_area=2, # do not call a 'spot' if it's area is below this threshold (measured in pixels)
    max_area=np.inf, # do not call a 'spot' if it's area is above this threshold (measured in pixels)
)

initial_spot_intensities, prop_results = psd.run(filtered_imgs)

spot_intensities = initial_spot_intensities.loc[initial_spot_intensities[Features.PASSES_THRESHOLDS]]